In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = None


In [2]:
# Parameters
product = {"nb": "/Users/thiago/workspace/duckDB/8.fastapi/movie_rec_system/products/eda-pipeline.ipynb"}


In [3]:
%pip install duckdb pandas numpy matplotlib requests python-dotenv jupysql duckdb-engine scikit-learn fastapi


Note: you may need to restart the kernel to use updated packages.


# EDA for Content Based Recommendation System

In [4]:
import os
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

# DuckDB database path:
# - In the Docker/Ploomber pipeline this file is created by the 'extract' task.
# - You can override it via env var DUCKDB_PATH.
_default_db = "movie_rec_system/movies_data.duckdb"
_fallback_db = "movies_data.duckdb"
DB_PATH = os.getenv("DUCKDB_PATH", _default_db if os.path.exists(_default_db) else _fallback_db)

con = duckdb.connect(DB_PATH)

def q(sql: str) -> pd.DataFrame:
    """Run a SELECT query and return a pandas DataFrame."""
    return con.sql(sql).df()


In [5]:
# Inspect schema
q("DESCRIBE movies")

,column_name,column_type,null,key,default,extra
0,genre_ids,INTEGER[],YES,None,None,None
1,id,INTEGER,YES,None,None,None
2,original_language,VARCHAR,YES,None,None,None
3,overview,VARCHAR,YES,None,None,None
4,popularity,DOUBLE,YES,None,None,None
5,release_date,TIMESTAMP,YES,None,None,None
6,title,VARCHAR,YES,None,None,None
7,vote_average,DOUBLE,YES,None,None,None
8,vote_count,INTEGER,YES,None,None,None


In [6]:
q("DESCRIBE genres")

,column_name,column_type,null,key,default,extra
0,id,INTEGER,YES,None,None,None
1,name,VARCHAR,YES,None,None,None


In [7]:
# Preview data
q("SELECT * FROM movies LIMIT 5")

,genre_ids,id,original_language,overview,popularity,release_date,title,vote_average,vote_count
0,"[28, 53, 80]",1306368,en,Trust frays when a team of Miami cops discover...,346.4826,2026-01-13,The Rip,7.000,845
1,"[16, 35, 12, 10751, 9648]",1084242,en,After cracking the biggest case in Zootopia's ...,388.6941,2025-11-26,Zootopia 2,7.596,1256
2,"[28, 14, 53]",1043197,en,Ten-year-old Aurora asks her hitman neighbor t...,288.1896,2025-12-11,Dust Bunny,6.564,128
3,"[28, 53, 80]",1326878,en,"Seeking revenge on her abusive husband, a woma...",279.0559,2024-08-16,Strangers,5.479,24
4,"[28, 878, 12]",1242898,en,"Cast out from his clan, a young Predator finds...",247.4768,2025-11-05,Predator: Badlands,7.755,1686


In [8]:
q("SELECT * FROM genres LIMIT 5")

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime


In [9]:
movie_genres = q("""
WITH ExpandedGenres AS (
    SELECT 
        m.id AS movie_id,
        mg.movie_genre_id,
        g.name AS genre_name
    FROM 
        (SELECT UNNEST(movies.genre_ids) AS movie_genre_id, movies.id FROM movies) AS mg
    JOIN movies m ON mg.id = m.id
    JOIN genres g ON mg.movie_genre_id = g.id
)
SELECT
    movie_id,
    STRING_AGG(genre_name, ', ') AS genre_names
FROM ExpandedGenres
GROUP BY movie_id;
""")
movie_genres.head()

,movie_id,genre_names
0,1054867,"Thriller, Crime, Action"
1,7451,"Action, Adventure, Thriller, Crime, Drama"
2,1315303,"Horror, Thriller"
3,617126,"Science Fiction, Adventure, Action, Science Fi..."
4,1151272,"Drama, Thriller, Music"


## Expanded genres per movie

(Computed above as `movie_genres`.)

Movies with 0 vote_counts are usually unreleased. Filtering this out allows for recommendations for only available movies.

In [10]:
# Materialize a helper table used for EDA plots
con.execute("""CREATE TABLE IF NOT EXISTS movie_genre_data AS
WITH ExpandedGenres AS (
    SELECT 
        m.id AS movie_id,
        mg.movie_genre_id,
        g.name AS genre_name
    FROM 
        (SELECT UNNEST(movies.genre_ids) as movie_genre_id, movies.id FROM movies) AS mg
    JOIN 
        movies m ON mg.id = m.id
    JOIN 
        genres g ON mg.movie_genre_id = g.id
),
genre_names AS (
    SELECT
    movie_id,
    STRING_AGG(genre_name, ', ') AS genre_names
FROM 
    ExpandedGenres
GROUP BY 
    movie_id
)
SELECT gn.genre_names, m.id, m.original_language,
       m.overview, m.popularity, m.release_date,
       m.title, m.vote_average, m.vote_count
FROM genre_names gn
JOIN movies m
ON gn.movie_id = m.id
WHERE m.vote_count != 0""")
q("SELECT COUNT(*) AS n_rows FROM movie_genre_data")

,n_rows
0,926


In [11]:
df = q("SELECT * FROM movie_genre_data")
df.head()

,genre_names,id,original_language,overview,popularity,release_date,title,vote_average,vote_count
0,"Thriller, Crime, Action",1054867,en,Washed-up revolutionary Bob exists in a state ...,73.5564,2025-09-23,One Battle After Another,7.422,2604
1,"Action, Adventure, Thriller, Crime, Drama",7451,en,Xander Cage is your standard adrenaline junkie...,73.2506,2002-08-09,xXx,5.982,4759
2,"Horror, Thriller",1315303,en,"Lucy, a college student, along with her friend...",48.1173,2026-01-01,Primate,6.600,61
3,"Science Fiction, Adventure, Action, Science Fi...",617126,en,Against the vibrant backdrop of a 1960s-inspir...,35.0764,2025-07-23,The Fantastic 4: First Steps,6.981,2904
4,"Drama, Thriller, Music",1151272,es,A man and his son arrive at a rave lost in the...,28.9640,2025-06-06,Sirāt,6.806,382


In [12]:
# plt.figure(figsize=(10,4))
# plt.hist(df["popularity"].dropna(), bins=20)
# plt.grid(True)
# plt.title("Movie Popularity")
# plt.xlabel("Popularity")
# plt.ylabel("Count")
# plt.tight_layout()
# plt.show()

In [13]:
# plt.figure(figsize=(10,4))
# plt.hist(df["vote_average"].dropna(), bins=20)
# plt.grid(True)
# plt.title("Vote Average")
# plt.xlabel("Vote Score")
# plt.ylabel("Count")
# plt.tight_layout()
# plt.show()

In [14]:
# plt.figure(figsize=(10,4))
# plt.hist(df["vote_count"].dropna(), bins=20)
# plt.grid(True)
# plt.title("Movie Vote Count")
# plt.xlabel("Votes")
# plt.ylabel("Count")
# plt.tight_layout()
# plt.show()

In [15]:
df["release_date"] = pd.to_datetime(df["release_date"])

df["release_year"] = df["release_date"].dt.year

movie_counts_by_year = df.groupby("release_year").size()

# plt.figure(figsize=(10, 6))
# plt.plot(movie_counts_by_year.index, movie_counts_by_year.values, marker="o")
# plt.xlabel("Release Year")
# plt.ylabel("Number of Movies Released")
# plt.title("Number of Movies Released Per Year")
# plt.tight_layout()
# plt.grid()
# plt.show()

In [16]:
df["genre_names"] = df["genre_names"].astype(str)


def get_genre_count(df):
    genre_counts = dict()

    for genres in df["genre_names"]:
        genre_list = genres.split(",")
        for genre in genre_list:
            if genre not in genre_counts:
                genre_counts[genre] = 1
            genre_counts[genre] += 1
    return genre_counts


genre_counts = get_genre_count(df)

sorted_genre_counts = dict(
    sorted(genre_counts.items(), key=lambda item: item[1], reverse=True)
)

genres = list(sorted_genre_counts.keys())
counts = list(sorted_genre_counts.values())

# plt.figure(figsize=(16, 7))
# plt.bar(genres, counts)
# plt.ylabel("Genres")
# plt.xlabel("Counts")
# plt.title("Genre Counts")
# plt.tight_layout()
# plt.xticks(rotation=60, ha="right")
# plt.grid()
# plt.show()

# Moving forward

Now that preliminary data wrangling and EDA has been accomplished, the next step would be to implement a content based recommendation system. 

We'll experiment using cosine TF-IDF on our movie's description and genres.